In [8]:
import requests
import csv
import re

# Canvas API configuration
api_url = "https://canvas.upenn.edu/api/v1/"
api_key = "25~wvL8FAPWrtCcUAcHBMmBkaDY8Q9mG4xMDemMFK784TPrXaJwRmeYEf8JHP4zhcBu"

# Header for requests
headers = {
    "Authorization": f"Bearer {api_key}"
}

# Course and assignment IDs
course_id = 1771351
assignment_id = 11924078  # Use the correct assignment ID for hw2

def get_students(course_id):
    endpoint = f"courses/{course_id}/students"
    url = f"{api_url}{endpoint}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve students. Status code: {response.status_code}, Response: {response.text}")
        return []

def get_assignment_comments(course_id, assignment_id, student_id):
    endpoint = f"courses/{course_id}/assignments/{assignment_id}/submissions/{student_id}?include[]=submission_comments"
    url = f"{api_url}{endpoint}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        submission = response.json()
        comments = submission.get('submission_comments', [])
        full_comment = " ".join(comment.get('comment', '') for comment in comments)
        return full_comment
    else:
        print(f"Failed to retrieve submission for student {student_id}. Status code: {response.status_code}, Response: {response.text}")
        return ""

def extract_grade(comment_text):
    # Enhanced RegEx to find 'Overall assignment grade' or 'Overall' followed by a grade
    # Stripping trailing spaces and underscores
    match = re.search(r"(Overall(?: Assignment Grade)?:?\s*)(CHECK(?:\sPLUS|\sMINUS)?|CHECK|GOOD)\s*[_]*", comment_text, re.IGNORECASE)
    if match:
        return match.group(2).strip().upper()  # Return corrected grade match
    return ""  # Return empty string when no grade is found


def create_gradebook_csv(course_id, assignment_name, csv_filename):
    students = get_students(course_id)
    with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['studentID', 'name', 'assignment', 'comment', 'grade'])
        
        for student in students:
            student_id = student['id']
            student_name = student['name']
            comment = get_assignment_comments(course_id, assignment_id, student_id)
            grade = extract_grade(comment)
            writer.writerow([student_id, student_name, assignment_name, comment, grade])

if __name__ == "__main__":
    create_gradebook_csv(course_id, 'hw2', 'gradebook_hw2.csv')
